# Deliverable 1: Retrieve Weather Data

In [21]:
# Import the dependencies.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from citipy import citipy
import gmaps
import requests
import time
from datetime import datetime

# Import the API keys.
import sys
sys.path.append('/Users/katiemtroll/Desktop/API_Keys')
from api_keys import key_google_maps
g_key = key_google_maps

import sys
sys.path.append('/Users/katiemtroll/Desktop/API_Keys')
from api_keys import key_open_weather
weather_key = key_open_weather

In [2]:
# Deliverable Objectives: 

# a. Create a new set of 2,000 random latitudes and longitudes.

# b. Get the nearest city using the citipy module.

# c. Perform an API call with the OpenWeatherMap.

# d. Retrieve the following information from the API call:
    # Latitude and longitude
    # Maximum temperature
    # Percent humidity
    # Percent cloudiness
    # Wind speed
    # Weather description (for example, clouds, fog, light rain, clear sky)
    
# e. Add the data to a new DataFrame.
    # The new cities_data DataFrame summarizes the current weather description.

# f. Export the DataFrame as a CSV file, and save it as WeatherPy_Database.csv in the Weather_Database folder.

In [3]:
# a. Create a new set of 2,000 random latitudes and longitudes.
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
coordinates

[(49.35936503074055, -55.33787660845968),
 (66.42986014930182, -74.94758578494002),
 (-66.34146987163561, -56.85576695115691),
 (-9.328260893337074, 168.32532713648794),
 (3.159766079077727, -77.05850259103441),
 (27.33919367449849, -135.14339605207198),
 (0.80340464955691, 161.51977837445486),
 (84.97795119346591, 59.69870899088326),
 (-18.30267952082282, 96.37408323523226),
 (19.583771225211805, -162.25750118244588),
 (-49.206879547840536, 147.77524016936172),
 (-9.83966981288458, 11.998390729416087),
 (78.79240136141496, 17.409356942826605),
 (57.399127976550744, 59.028668183004214),
 (-63.894041388675525, -59.56620088165646),
 (15.10846774874085, 159.91315772738375),
 (43.76800834519335, -80.6045895636955),
 (-29.01226688816896, -22.79432716501813),
 (62.71543822292284, -142.77186787990865),
 (86.02987557168083, 144.73358343333217),
 (20.457929439702582, -89.69894786318972),
 (42.89303589432396, -76.93080550130557),
 (31.00808576057574, -161.87339190786304),
 (-28.8944044369634, 15

In [5]:
# b. Get the nearest cities using citipy
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

763

In [6]:
# create the weather url
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_key

In [7]:
# c. Perform an API call on OpenWeather

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
# enumerate to get the index in a way that can log in the list
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    # time sleep to prevent time-out error (only 60 calls per minute allowed on OpenWeather free tier)
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_weather_descrip = city_weather["weather"][0]["description"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Weather Description": city_weather_descrip,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | botwood
Processing Record 2 of Set 1 | iqaluit
Processing Record 3 of Set 1 | ushuaia
Processing Record 4 of Set 1 | lata
Processing Record 5 of Set 1 | suarez
Processing Record 6 of Set 1 | pacific grove
Processing Record 7 of Set 1 | buala
Processing Record 8 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 9 of Set 1 | labuhan
Processing Record 10 of Set 1 | kapaa
Processing Record 11 of Set 1 | hobart
Processing Record 12 of Set 1 | luanda
Processing Record 13 of Set 1 | longyearbyen
Processing Record 14 of Set 1 | shalya
Processing Record 15 of Set 1 | butaritari
Processing Record 16 of Set 1 | arthur
Processing Record 17 of Set 1 | vila velha
Processing Record 18 of Set 1 | fairbanks
Processing Record 19 of Set 1 | chokurdakh
Processing Record 20 of Set 1 | muna
Processing Record 21 of Set 1 | geneva
Processing Record 22 of Set 1 | oranjemund
Processing Record 23 of Se

Processing Record 38 of Set 4 | ngukurr
City not found. Skipping...
Processing Record 39 of Set 4 | port hardy
Processing Record 40 of Set 4 | coihaique
Processing Record 41 of Set 4 | pangnirtung
Processing Record 42 of Set 4 | diapaga
Processing Record 43 of Set 4 | novaya lyalya
Processing Record 44 of Set 4 | bubaque
Processing Record 45 of Set 4 | qaanaaq
Processing Record 46 of Set 4 | shenzhen
Processing Record 47 of Set 4 | leningradskiy
Processing Record 48 of Set 4 | bongandanga
Processing Record 49 of Set 4 | nguiu
City not found. Skipping...
Processing Record 50 of Set 4 | pevek
Processing Record 1 of Set 5 | calvinia
Processing Record 2 of Set 5 | mecca
Processing Record 3 of Set 5 | namtsy
Processing Record 4 of Set 5 | san jeronimo
Processing Record 5 of Set 5 | oistins
Processing Record 6 of Set 5 | marcona
City not found. Skipping...
Processing Record 7 of Set 5 | tsihombe
City not found. Skipping...
Processing Record 8 of Set 5 | narsaq
Processing Record 9 of Set 5 | 

Processing Record 24 of Set 8 | san patricio
Processing Record 25 of Set 8 | ialibu
Processing Record 26 of Set 8 | catuday
Processing Record 27 of Set 8 | ranot
Processing Record 28 of Set 8 | half moon bay
Processing Record 29 of Set 8 | llata
Processing Record 30 of Set 8 | uzhur
Processing Record 31 of Set 8 | illapel
Processing Record 32 of Set 8 | rawannawi
City not found. Skipping...
Processing Record 33 of Set 8 | kassala
Processing Record 34 of Set 8 | chagda
City not found. Skipping...
Processing Record 35 of Set 8 | vitim
Processing Record 36 of Set 8 | warri
Processing Record 37 of Set 8 | matina
Processing Record 38 of Set 8 | hunterville
Processing Record 39 of Set 8 | jerome
Processing Record 40 of Set 8 | tirano
Processing Record 41 of Set 8 | raga
Processing Record 42 of Set 8 | padang
Processing Record 43 of Set 8 | necochea
Processing Record 44 of Set 8 | mount pleasant
Processing Record 45 of Set 8 | kendal
Processing Record 46 of Set 8 | inuvik
Processing Record 47

Processing Record 12 of Set 12 | ewa beach
Processing Record 13 of Set 12 | tel aviv-yafo
Processing Record 14 of Set 12 | buin
Processing Record 15 of Set 12 | ust-karsk
Processing Record 16 of Set 12 | nesbyen
Processing Record 17 of Set 12 | salalah
Processing Record 18 of Set 12 | marienburg
Processing Record 19 of Set 12 | sambava
Processing Record 20 of Set 12 | talcahuano
Processing Record 21 of Set 12 | bowen
Processing Record 22 of Set 12 | gisborne
Processing Record 23 of Set 12 | kloulklubed
Processing Record 24 of Set 12 | kifri
Processing Record 25 of Set 12 | dong hoi
Processing Record 26 of Set 12 | charters towers
Processing Record 27 of Set 12 | roma
Processing Record 28 of Set 12 | kavaratti
Processing Record 29 of Set 12 | lisburn
Processing Record 30 of Set 12 | lerwick
Processing Record 31 of Set 12 | erzincan
Processing Record 32 of Set 12 | barda
Processing Record 33 of Set 12 | cap malheureux
Processing Record 34 of Set 12 | ust-nera
Processing Record 35 of Set 

Processing Record 50 of Set 15 | langres
Processing Record 1 of Set 16 | chivilcoy
Processing Record 2 of Set 16 | gambela
Processing Record 3 of Set 16 | virginia beach
Processing Record 4 of Set 16 | pizarro
Processing Record 5 of Set 16 | homer
Processing Record 6 of Set 16 | mahibadhoo
Processing Record 7 of Set 16 | ostrowiec swietokrzyski
Processing Record 8 of Set 16 | makinsk
Processing Record 9 of Set 16 | havelock
Processing Record 10 of Set 16 | tuy hoa
Processing Record 11 of Set 16 | dallas
Processing Record 12 of Set 16 | rondonopolis
Processing Record 13 of Set 16 | dawei
-----------------------------
Data Retrieval Complete      
-----------------------------


In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Weather Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Botwood,49.1499,-55.3482,overcast clouds,22.89,73,97,13.11,CA,2022-01-05 14:30:08
1,Iqaluit,63.7506,-68.5145,light snow,2.93,84,90,24.00,CA,2022-01-05 14:20:16
2,Ushuaia,-54.8000,-68.3000,few clouds,53.22,62,20,28.77,AR,2022-01-05 14:15:18
3,Lata,40.1629,-8.3327,light rain,54.91,95,100,10.31,PT,2022-01-05 14:30:08
4,Suarez,2.9540,-76.6964,overcast clouds,73.60,83,93,0.96,CO,2022-01-05 14:30:09
5,Pacific Grove,36.6177,-121.9166,mist,51.58,94,75,1.01,US,2022-01-05 14:30:10
6,Buala,-8.1450,159.5921,light rain,79.90,84,88,7.38,SB,2022-01-05 14:30:10
7,Labuhan,-6.8844,112.2051,overcast clouds,78.35,76,100,7.96,ID,2022-01-05 14:30:11
8,Kapaa,22.0752,-159.3190,clear sky,66.52,78,1,6.91,US,2022-01-05 14:30:11
9,Hobart,-42.8794,147.3294,overcast clouds,59.25,75,90,9.22,AU,2022-01-05 14:22:39


In [18]:
# Change Order
# new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Weather Description"]
# city_data_df = city_data_df[new_column_order]
# Change weather description name
# city_data_df = city_data_df.rename(columns={"Weather Description": "Current Description"})
# Drop Date
#city_data_df = city_data_df.drop(columns={"Date"})
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Botwood,CA,49.1499,-55.3482,22.89,73,97,13.11,overcast clouds
1,Iqaluit,CA,63.7506,-68.5145,2.93,84,90,24.00,light snow
2,Ushuaia,AR,-54.8000,-68.3000,53.22,62,20,28.77,few clouds
3,Lata,PT,40.1629,-8.3327,54.91,95,100,10.31,light rain
4,Suarez,CO,2.9540,-76.6964,73.60,83,93,0.96,overcast clouds
5,Pacific Grove,US,36.6177,-121.9166,51.58,94,75,1.01,mist
6,Buala,SB,-8.1450,159.5921,79.90,84,88,7.38,light rain
7,Labuhan,ID,-6.8844,112.2051,78.35,76,100,7.96,overcast clouds
8,Kapaa,US,22.0752,-159.3190,66.52,78,1,6.91,clear sky
9,Hobart,AU,-42.8794,147.3294,59.25,75,90,9.22,overcast clouds


In [19]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")